# DE2-BI-Project - ETL Process

## Importing libraries

In [19]:
from sqlalchemy import text

import pandas as pd

In [20]:
from config.settings import MARIA_DB_URI, MS_SQL_URI

import python.database_api as db

## Instantiate database connections

### MariaDB

In [21]:
mariadb_session_manager = db.SessionManager(MARIA_DB_URI)

try:
    mariadb_session = mariadb_session_manager.get_session()
    print("MariaDB connection successful")
    mariadb_session.close()
except Exception as e:
    print(e)
    

MariaDB connection successful


Drop mariadb tables

In [22]:
mariadb_session_manager.get_session().execute(text("DROP TABLE IF EXISTS sales;"))

mariadb_session_manager.get_session().execute(text("DROP TABLE IF EXISTS sales_detail;"))

mariadb_session_manager.get_session().execute(text("DROP TABLE IF EXISTS customer;"))

mariadb_session_manager.get_session().execute(text("DROP TABLE IF EXISTS sales_reason;"))

mariadb_session_manager.get_session().execute(text("DROP TABLE IF EXISTS special_offer;"))

mariadb_session_manager.get_session().execute(text("DROP TABLE IF EXISTS credit_card;"))

mariadb_session_manager.get_session().commit()

mariadb_session_manager.get_session().close()

### MsSQL

In [23]:
ms_sql_session_manager = db.SessionManager(MS_SQL_URI)

try:
    ms_sql_session = ms_sql_session_manager.get_session()
    print("MsSQL connection successful")
    ms_sql_session.close()
except Exception as e:
    print(e)

MsSQL connection successful


## Extract Transform Load (ETL)

### Sales Table

#### Extract

In [24]:
sales_sql = """
SELECT
    sales.SalesOrderID,
    OrderDate,
    DueDate,
    ShipDate,
    OnlineOrderFlag,
    SalesOrderNumber,
    PurchaseOrderNumber,
    AccountNumber,
    CustomerID,
    SalesPersonID,
    TerritoryID,
    CurrencyRateID,
    SubTotal,
    TaxAmt,
    Freight,
    TotalDue,
    CreditCardID,
    SalesReasonID
FROM Sales.SalesOrderHeader as sales
INNER JOIN Sales.SalesOrderHeaderSalesReason as reason
    ON sales.SalesOrderID = reason.SalesOrderID;
"""

df = pd.read_sql(sales_sql, ms_sql_session_manager.get_engine())

df.head()

,SalesOrderID,OrderDate,DueDate,ShipDate,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,CreditCardID,SalesReasonID
0,43697,2011-05-31,2011-06-12,2011-06-07,True,SO43697,None,10-4030-021768,21768,None,6,4.0,3578.27,286.2616,89.4568,3953.9884,4319.0,5
1,43697,2011-05-31,2011-06-12,2011-06-07,True,SO43697,None,10-4030-021768,21768,None,6,4.0,3578.27,286.2616,89.4568,3953.9884,4319.0,9
2,43702,2011-06-01,2011-06-13,2011-06-08,True,SO43702,None,10-4030-027645,27645,None,4,NaN,3578.27,286.2616,89.4568,3953.9884,8087.0,5
3,43702,2011-06-01,2011-06-13,2011-06-08,True,SO43702,None,10-4030-027645,27645,None,4,NaN,3578.27,286.2616,89.4568,3953.9884,8087.0,9
4,43703,2011-06-01,2011-06-13,2011-06-08,True,SO43703,None,10-4030-016624,16624,None,9,15.0,3578.27,286.2616,89.4568,3953.9884,9497.0,5


#### Transform

#### Load

In [25]:
df.to_sql(
    'sales', 
    mariadb_session_manager.get_engine(), 
    if_exists='replace', 
    index=False
)

27647

### SalesDetail Table

In [26]:
sales_detail_sql = """
SELECT SalesOrderID,
    SalesOrderDetailID,
    CarrierTrackingNumber,
    OrderQty,
    ProductID,
    SpecialOfferID,
    UnitPrice,
    UnitPriceDiscount,
    LineTotal
       FROM Sales.SalesOrderDetail;
"""

df = pd.read_sql(sales_detail_sql, ms_sql_session_manager.get_engine())

df.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal
0,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994
1,43659,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982
2,43659,3,4911-403C-98,1,778,1,2024.994,0.0,2024.994
3,43659,4,4911-403C-98,1,771,1,2039.994,0.0,2039.994
4,43659,5,4911-403C-98,1,772,1,2039.994,0.0,2039.994


#### Transform

#### Load

In [27]:
df.to_sql(
    'sales_detail', 
    mariadb_session_manager.get_engine(), 
    if_exists='replace', 
    index=False
)

121317

### Customer Table
#### Extract

In [28]:
customer_sql = """
SELECT
    CustomerID,
    PersonID,
    StoreID,
    TerritoryID,
    AccountNumber
    From Sales.Customer;
"""

df = pd.read_sql(customer_sql, ms_sql_session_manager.get_engine())

df.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber
0,1,NaN,934.0,1,AW00000001
1,2,NaN,1028.0,1,AW00000002
2,3,NaN,642.0,4,AW00000003
3,4,NaN,932.0,4,AW00000004
4,5,NaN,1026.0,4,AW00000005


#### Transform

#### Load

In [29]:
df.to_sql(
    'customer', 
    mariadb_session_manager.get_engine(), 
    if_exists='replace', 
    index=False
)

19820

### SalesReason Table
#### Extract

In [30]:
sales_reason_sql = """
SELECT
    SalesReasonID,
    Name,
    ReasonType
    From Sales.SalesReason;
"""

df = pd.read_sql(sales_reason_sql, ms_sql_session_manager.get_engine())

df.head()

,SalesReasonID,Name,ReasonType
0,1,Price,Other
1,2,On Promotion,Promotion
2,3,Magazine Advertisement,Marketing
3,4,Television Advertisement,Marketing
4,5,Manufacturer,Other


#### Transform

#### Load

In [31]:
df.to_sql(
    'sales_reason', 
    mariadb_session_manager.get_engine(), 
    if_exists='replace', 
    index=False
)

10

### SpecialOffer Table
#### Extract

In [32]:
special_offer_sql = """
SELECT
    so.SpecialOfferID,
    ProductID,
    Description,
    DiscountPct,
    Type,
    Category,
    StartDate,
    EndDate,
    MinQty,
    MaxQty
    From Sales.SpecialOfferProduct as sop
    INNER JOIN Sales.SpecialOffer as so
        ON sop.SpecialOfferID = so.SpecialOfferID;
"""

df = pd.read_sql(special_offer_sql, ms_sql_session_manager.get_engine())

df.head()

,SpecialOfferID,ProductID,Description,DiscountPct,Type,Category,StartDate,EndDate,MinQty,MaxQty
0,1,680,No Discount,0.0,No Discount,No Discount,2011-05-01,2014-11-30,0,NaN
1,1,706,No Discount,0.0,No Discount,No Discount,2011-05-01,2014-11-30,0,NaN
2,1,707,No Discount,0.0,No Discount,No Discount,2011-05-01,2014-11-30,0,NaN
3,1,708,No Discount,0.0,No Discount,No Discount,2011-05-01,2014-11-30,0,NaN
4,1,709,No Discount,0.0,No Discount,No Discount,2011-05-01,2014-11-30,0,NaN


#### Transform

#### Load

In [33]:
df.to_sql(
    'special_offer', 
    mariadb_session_manager.get_engine(), 
    if_exists='replace', 
    index=False
)

538

### CreditCard Table
#### Extract

In [34]:
credit_card_sql = """

SELECT
    CreditCardID,
    CardType,
    CardNumber,
    ExpMonth,
    ExpYear
    from Sales.CreditCard;
"""

df = pd.read_sql(credit_card_sql, ms_sql_session_manager.get_engine())

df.head()

,CreditCardID,CardType,CardNumber,ExpMonth,ExpYear
0,1,SuperiorCard,33332664695310,11,2006
1,2,Distinguish,55552127249722,8,2005
2,3,ColonialVoice,77778344838353,7,2005
3,4,ColonialVoice,77774915718248,7,2006
4,5,Vista,11114404600042,4,2005


#### Transform

#### Load

In [35]:
df.to_sql(
    'credit_card', 
    mariadb_session_manager.get_engine(), 
    if_exists='replace', 
    index=False
)

19118